In [1]:
# Install Hugging Face Transformers and Datasets
!pip install transformers datasets
!pip install torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [3]:
!pip install transformers datasets


In [4]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


In [5]:
from datasets import load_dataset

# Load a sample dataset for text summarization
dataset = load_dataset("cnn_dailymail", split="train[:1%]")

# Preprocess the data (tokenization)
def tokenize_function(examples):
    return tokenizer(examples['article'], truncation=True, padding="max_length", max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

ValueError: Config name is missing.
Please pick one among the available configs: ['1.0.0', '2.0.0', '3.0.0']
Example of usage:
	`load_dataset('cnn_dailymail', '1.0.0')`

In [6]:
from datasets import load_dataset

# Load a sample dataset for text summarization, specifying the config name
dataset = load_dataset("cnn_dailymail", name="3.0.0", split="train[:1%]") # Added name="3.0.0" to specify the dataset version

# Preprocess the data (tokenization)
def tokenize_function(examples):
    return tokenizer(examples['article'], truncation=True, padding="max_length", max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Map:   0%|          | 0/2871 [00:00<?, ? examples/s]

ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [7]:
from datasets import load_dataset

# Load a sample dataset for text summarization, specifying the config name
dataset = load_dataset("cnn_dailymail", name="3.0.0", split="train[:1%]") # Added name="3.0.0" to specify the dataset version

# Preprocess the data (tokenization)
def tokenize_function(examples):
    # Assign the EOS token as the padding token
    tokenizer.pad_token = tokenizer.eos_token
    return tokenizer(examples['article'], truncation=True, padding="max_length", max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/2871 [00:00<?, ? examples/s]

In [10]:
# Define a function for generating text with a given prompt
def generate_text(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", max_length=150, truncation=True) # Truncate the input to 150 tokens
    outputs = model.generate(inputs['input_ids'], max_new_tokens=50, num_return_sequences=1) # Use max_new_tokens instead of max_length
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

prompt = "Summarize the following article: " + dataset[0]['article']
print(generate_text(prompt))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Summarize the following article: LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I'm not going to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar." The actor, who has been in the news for his role in the hit film "Harry Potter


In [11]:
# Different prompt variations for comparison
prompts = [
    "Summarize the following article briefly: ",
    "Can you provide a detailed summary of the following article? ",
    "Write a concise summary of this article: "
]

for p in prompts:
    print(f"Prompt: {p}\nGenerated Summary: {generate_text(p + dataset[0]['article'])}\n")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Summarize the following article briefly: 
Generated Summary: Summarize the following article briefly: LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I'm not going to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar." The actor, who has been in the news for his role in the hit film "Harry



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Can you provide a detailed summary of the following article? 
Generated Summary: Can you provide a detailed summary of the following article? LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I'm not going to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar." The actor, who has been in the news for his ro

In [12]:
# Simulate feedback loop (just an example)
human_feedback = 4  # Rating the output on a scale of 1 to 5
if human_feedback < 3:
    print("Consider revising the prompt for better output.")
else:
    print("Output is satisfactory.")


Output is satisfactory.


In [13]:
# Save the fine-tuned model
model.save_pretrained("fine_tuned_model")
tokenizer.save_pretrained("fine_tuned_tokenizer")


('fine_tuned_tokenizer/tokenizer_config.json',
 'fine_tuned_tokenizer/special_tokens_map.json',
 'fine_tuned_tokenizer/vocab.json',
 'fine_tuned_tokenizer/merges.txt',
 'fine_tuned_tokenizer/added_tokens.json')